In [1]:
from Environments.StopLoss import StopLossMarket
import numpy as np
import matplotlib.pyplot as plt
import ray
import gym
from ray.tune.registry import register_env
from ray.rllib.agents.ppo.ddppo import DEFAULT_CONFIG
from ray.rllib.agents.ppo.ddppo import DDPPOTrainer
import os
import gc

In [2]:

ray.init()
trainer_config = DEFAULT_CONFIG.copy()
trainer_config['model']['use_lstm'] = False
trainer_config['model']['lstm_cell_size'] = 16
trainer_config['num_gpus'] = 0
trainer_config['num_gpus_per_worker'] = 1
trainer_config['num_envs_per_worker'] = 8
trainer_config['gamma'] = 0.99
trainer_config['entropy_coeff'] = 0.00
trainer_config['framework'] = 'torch'
trainer_config['num_workers'] = 1
trainer_config['horizon'] = 9999
trainer_config['rollout_fragment_length'] = 9999
trainer_config['model']['framestack'] = False
trainer_config['model']['fcnet_hiddens'] = [128, 128, 128]
trainer_config['lr'] = 15e-05
trainer = DDPPOTrainer(trainer_config, StopLossMarket)
best_reward = -np.inf
trainer.save()
hall_of_fame = [0]
last_checkpoint = 0


2020-10-14 19:47:59,115	INFO services.py:1164 -- View the Ray dashboard at http://127.0.0.1:8266
2020-10-14 19:47:59,121	WARNING services.py:1617 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 66965504 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.
2020-10-14 19:48:00,357	ERROR syncer.py:63 -- Log sync requires rsync to be installed.
2020-10-14 19:48:00,363	INFO trainer.py:616 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/root/miniconda3/envs/rl/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=13742) /root/

In [ ]:
%%time
for i in range(40):
    print("Training iteration {}...".format(i))
    results = trainer.train()
    this_reward = results['episode_reward_max']
    avg_reward = results['episode_reward_mean']
    if this_reward > best_reward:
        best_reward = this_reward
        trainer.save()
        path = trainer.logdir + 'checkpoint_{0}/checkpoint-{0}'.format(last_checkpoint)
        os.remove(path)
        last_checkpoint = i + 1
        hall_of_fame.append(i+1)
        print('New best reward') 
        print(best_reward)
    if i % 10 == 0:
        print('Best Reward So Far')
        print(best_reward)
        print('Average Rewards')
        print(avg_reward)


(pid=13742) /root/miniconda3/envs/rl/lib/python3.8/site-packages/ray/rllib/utils/torch_ops.py:65: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629395347/work/torch/csrc/utils/tensor_numpy.cpp:141.)
(pid=13742)   tensor = torch.from_numpy(np.asarray(item))


Training iteration 0...


In [ ]:
env = StopLossMarket(episode_length = 10000)

In [ ]:
env.reset()
for i in range(9999):
    action = np.random.randint(2)
    state, reward, done, info = env.step(action)
    if state[-1] != 0:
        print(state[-1] * 2000)